El método de asignación y desasignación de memoria de Python es automático. 

El usuario no tiene que preasignar o desasignar memoria de forma similar al uso de la asignación de memoria dinámica en lenguajes como C o C++. 
Python utiliza dos estrategias para la asignación de memoria: 

- Recuento de referencias
- Recolección de basura

Antes de la versión 2.0 de Python, el intérprete de Python solo usaba el conteo de referencias para la administración de la memoria. El recuento de referencias funciona contando el número de veces que otros objetos del sistema hacen referencia a un objeto. Cuando se eliminan las referencias a un objeto, se reduce el recuento de referencias de un objeto. Cuando el recuento de referencias llega a cero, el objeto se desasigna.

**Variables en Memoria**

Podemos encontrar la dirección de memoria a la que hace referencia una variable, utilizando la función id().

La función id() devuelve la dirección de memoria de su argumento como un número entero de base-10.

Podemos utilizar la función hex() para convertir el número de base-10 a base-16.

In [1]:
my_var = 10
print('my_var = {0}'.format(my_var))
print('memory address of my_var (decimal): {0}'.format(id(my_var)))
print('memory address of my_var (hex): {0}'.format(hex(id(my_var))))

my_var = 10
memory address of my_var (decimal): 3046026603088
memory address of my_var (hex): 0x2c535556a50


In [2]:
greeting = 'Hello'
print('greeting = {0}'.format(greeting))
print('memory address of my_var (decimal): {0}'.format(id(greeting)))
print('memory address of my_var (hex): {0}'.format(hex(id(greeting))))

greeting = Hello
memory address of my_var (decimal): 3046113691184
memory address of my_var (hex): 0x2c53a864630


Observe cómo la dirección de memoria de mi_var es diferente a la de saludo.

Estrictamente hablando, mi_var no es "igual" a 10.

En cambio, mi_var es una referencia a un objeto (entero) (que contiene el valor 10) situado en la dirección de memoria id(mi_var)

Lo mismo ocurre con la variable saludo.

**Recuento de referencias**

Veremos qué sucede detrás de escenas cuando asignamos un valor a una variable.

In [33]:
#En este punto ambas variables tienen asignado el mismo valor, como es de esperar. 
#A continuación, vemos qué sucede cuando se asignan listas en vez de números a estas variables.
a = [1, 2, 3, 4, 5, 6]
b = a
a[2] = 10
print(a)
print(b)

#¿Cómo puede ser esto? La respuesta es simple. 
#Cuando asignamos una lista a una variable, lo que guardamos en la misma es en realidad una referencia a la lista y no la lista en sí.
#Como consecuencia, al copiar la variable a otra lo que hicimos fue copiar la referencia y no un valor.
#De esta manera, al modificar la lista referenciada en a el cambio es también visible en b.

[1, 2, 10, 4, 5, 6]
[1, 2, 10, 4, 5, 6]


In [38]:
import copy
#Finalmente veamos cómo podemos hacer que la referencia a un elemento mutable no se copie a otro en el escenario anterior.
#Es decir, los cambios que se hagan a una variable no se verán reflejados en la otra. 
#Para eso utilizaremos la función deepcopy() del módulo copy como vemos a continuación:

a = [1, 2, 3, 4, 5, 6]
b = copy.deepcopy(a)
a[2] = 10
print(a)
print(b)

[1, 2, 10, 4, 5, 6]
[1, 2, 3, 4, 5, 6]


In [35]:
print('memory address of my_var (decimal): {0}'.format(id(a)))
print('memory address of my_var (hex): {0}'.format(hex(id(a))))

print('memory address of my_var (decimal): {0}'.format(id(b)))
print('memory address of my_var (hex): {0}'.format(hex(id(b))))

memory address of my_var (decimal): 3046142398592
memory address of my_var (hex): 0x2c53c3c5080
memory address of my_var (decimal): 3046142069888
memory address of my_var (hex): 0x2c53c374c80


In [40]:
import ctypes

#Conteo de cuantas veces a sido referenciado.

def ref_count(address):
    return ctypes.c_long.from_address(address).value

ref_count(id(a))

1

In [39]:
#Hay otra función integrada que podemos usar para obtener el recuento de referencia:

import sys
sys.getrefcount(a)

#Pero, ¿por qué devuelve 2, en lugar del esperado 1 que obtuvimos con la función anterior?
#Respuesta: La función sys.getrefcount() toma my_var como argumento, lo que significa que también recibe (y almacena) una referencia a la
#dirección de memoria de my_var; por lo tanto, el conteo está desfasado en 1.
#Por lo tanto, usaremos from_address() en su lugar.

#Hacemos otra referencia a la misma referencia que my_var:

2

In [41]:
other_var = my_var

#Veamos la dirección de memoria de esas dos variables y los recuentos de referencia:

print(hex(id(my_var)), hex(id(other_var)))
print(ref_count(id(my_var)))

0x2c53c3d0480 0x2c53c3d0480
2


In [42]:
#Forzar una referencia para que desaparezca:
other_var = None

print(ref_count(id(my_var)))

#Probablemente nunca necesites hacer algo como esto en Python. 
#La administración de la memoria es completamente transparente: esto es solo para ilustrar algo de lo que sucede detrás de escena,
#ya que ayuda a comprender los conceptos futuros.

1


**Recolección de Basura**

Usamos la misma función que usamos en la lección sobre el conteo de referencias para calcular el número de referencias a un objeto específico (usando su dirección de memoria para evitar crear una referencia extra)

In [45]:
import ctypes
import gc

def ref_count(address):
    return ctypes.c_long.from_address(address).value

#Creamos una función que buscará los objetos en el GC por una identificación específica y nos dirá si el objeto fue encontrado o no:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not found"

A continuación definimos dos clases que usaremos para crear una referencia circular

El constructor de la clase A creará una instancia de la clase B y se la pasará al constructor de la clase B que luego almacenará esa referencia en alguna variable de instancia.

In [46]:
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b:{1}'.format(hex(id(self)), hex(id(self.b))))
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

#Apagamos el GC para que podamos ver cómo se ven afectados los recuentos de referencia cuando el GC no se ejecuta y cuando lo hace (ejecutándolo manualmente).
gc.disable()

#Ahora creamos una instancia de A, que, a su vez, creará una instancia de B que almacenará una referencia a la instancia de llamada A.
my_var = A()

B: self: 0x2c53b9fbd90, a: 0x2c53ba16280
A: self: 0x2c53ba16280, b:0x2c53b9fbd90


In [47]:
#Como podemos ver, se ejecutaron los constructores de A y B, y también vemos en las direcciones de memoria que tenemos una referencia circular.
#De hecho, my_var también es una referencia a la misma instancia A:
print(hex(id(my_var)))

0x2c53ba16280


In [48]:
#Otra forma de ver esto:
print('a: \t{0}'.format(hex(id(my_var))))
print('a.b: \t{0}'.format(hex(id(my_var.b))))
print('b.a: \t{0}'.format(hex(id(my_var.b.a))))

a: 	0x2c53ba16280
a.b: 	0x2c53b9fbd90
b.a: 	0x2c53ba16280


In [50]:
a_id = id(my_var)
b_id = id(my_var.b)

#Podemos ver cuántas referencias tenemos para a y b:

print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

#Como podemos ver, la instancia A tiene dos referencias (una de my_var, la otra de la variable de instancia b en la instancia B)

#La instancia B tiene una referencia (de la variable de instancia A a)

#Ahora, eliminemos la referencia a la instancia A que está en manos de my_var:

refcount(a) = 2
refcount(b) = 1
a: Object exists
b: Object exists


In [51]:
my_var= None

print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 1
refcount(b) = 1
a: Object exists
b: Object exists


In [52]:
#Como podemos ver, los conteos de referencia ahora son ambos iguales a 1 (una referencia circular pura),
#y el conteo de referencia por sí solo no destruyó A y B. 

#casos - todavía están alrededor. Si no se realiza una recolección de elementos no utilizados, se produciría una fuga de memoria.

#Ejecutemos el GC manualmente y volvamos a verificar si los objetos aún existen:

gc.collect()
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 0
refcount(b) = 0
a: Not found
b: Not found
